In [1]:
# -*- coding: utf-8 -*-
# @Time    : 2025/04/03 
import os
import re
import emoji
from zhon.hanzi import punctuation as cn_punc
import json
import glob
from langid import classify
import jieba 

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Load the MeetFresh user dictionary
jieba.load_userdict("MF_dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Han\AppData\Local\Temp\jieba.cache
Loading model cost 0.738 seconds.
Prefix dict has been built successfully.


In [ ]:
# Define the constants

core_keywords = {
    '鲜芋仙', 'Meet Fresh', 'MeetFresh', '台湾美食', '甜品', 
    '芋圆', 'taro', '仙草', 'grass jelly', '奶茶', 'milk tea',
    '豆花', 'tofu pudding', '奶刨冰', 'milked shaved ice',
    '红豆汤', 'purple rice soup', '紫米粥', 'red bean soup',
    '2001 Coit Rd', 'Park Pavillion Center', '(972) 596-6088'
}


In [4]:
cont = pd.read_json('..\..\Data\processed\contents_cooked.json')

In [5]:
cont.head(2)

,note_id,user_id,title,note_body,tag_list,image_count,content_type_video,hot_note,post_time,last_update_time,scraped_time,elapsed_time,liked_count,collected_count,comment_count,share_count,interaction_count
0,67d0d605000000002903d86a,576d3bde82ec3952ff40c5e1,有没有和我一样【吃茶三千】一喝一个不吱声的,连地址都不想写了[笑哭R]\n虽然环境还不错，颜值也不错\n但我感觉自己完全拔草了\n尝了m...,"达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店",1,0,0,2025-03-12 00:32:05,2025-03-12 00:32:06,2025-03-12 00:55:08,0,5,2,10,3,17
1,67c8960f000000002503ff5f,576d3bde82ec3952ff40c5e1,【达拉斯·玩】超治愈系手工体验！MUMU Garden,"📍301 W Parker Rd #208[话题]#, Plano, TX 75023（快乐...","208,达拉斯生活,达拉斯探店,达拉斯,达拉斯吃喝玩乐,达拉斯手工,达拉斯周末,达拉斯周边,...",14,0,0,2025-03-05 18:21:03,2025-03-05 18:21:04,2025-03-12 00:55:08,6,47,23,7,37,77


In [6]:
# combine the title and note_body into a single string
def process_text(note):
    return note['title'] + ' ' + note['note_body']

# Apply the function to the DataFrame
cont['text'] = cont.apply(process_text, axis=1).astype(str)


In [7]:
# slice the 1st row of cont
cont.iloc[11]

note_id                                        6722d61c000000001a01e79f
user_id                                        576d3bde82ec3952ff40c5e1
title                                            【达拉斯·吃】刘一手，目前火锅届性价比天花板
note_body             📍2001 Coit Rd Ste 137D, Plano, TX 75075\n听说新团队...
tag_list              北美刘一手,达拉斯火锅,达拉斯美食,达拉斯生活,达拉斯,达拉斯刘一手,达拉斯探店,达拉斯周边...
image_count                                                          15
content_type_video                                                    0
hot_note                                                              1
post_time                                           2024-10-31 00:58:04
last_update_time                                    2024-10-31 12:52:24
scraped_time                                        2025-03-12 00:55:08
elapsed_time                                                        131
liked_count                                                          93
collected_count                                                 

中文文本清洗黄金四步法

In [ ]:
def fullwidth_to_halfwidth(text:str) -> str:
    """全角转半角（保留￥符号）"""
    translation_table = str.maketrans({
        '！': '!', '“': '"', '”': '"', '‘': "'", '’': "'",
        '、': ',', '，': ',', '；': ';', '：': ':', '？': '?',
        '《': '<', '》': '>', '【': '[', '】': ']', '·': '.',
        '～': '~', '—': '-', '（': '(', '）': ')', '　': ' '
    })
    return text.translate(translation_table)

def normalize_punctuation(text:str) -> str:
    """符号标准化（保留emoji, retain 字符的位置信息但牺牲了效率, 之后可以考虑优化"""
    # 定义保留符号集（新增%$￥）
    keep_symbols = {"'", '"', ',', '.', ':', ';', '!', '?', '-', 
                   '(', ')', '<', '>', '[', ']', '&', '#', '@',
                   '%', '$', '￥', '/', '=', '+', '~', '^'}
    
    # 字符级处理, 
    cleaned_chars = []
    for char in text:
        # 保留条件：字母数字/汉字/keep_symbols/emoji
        if (char.isalnum() or
            '\u4e00' <= char <= '\u9fff' or
            char in keep_symbols or
            emoji.is_emoji(char)):
            cleaned_chars.append(char)
        else:
            cleaned_chars.append(' ')
    
    return ''.join(cleaned_chars)

def remove_urls(text:str) -> str:
    """适配中文域名的URL移除"""
    url_pattern = re.compile(
        r'(?:https?://)?(?:[a-zA-Z0-9\u4e00-\u9fff-]+\.)+[a-zA-Z]{2,}'
        r'(?:/\S*)?(?:\?\S*)?(?:#\S*)?',
        flags=re.IGNORECASE
    )
    return url_pattern.sub('', text)

def basic_clean(
        text : str
    ) -> str:
    """
    对文本进行基础层清洗，包括去除HTML标签、URL、特殊符号处理, 空格标准化等。
    1. 移除HTML标签
    2. 移除URL（适配中文域名）
    3. 处理特殊符号（保留常用符号和emoji, 全角标点转半角）
    4. 标准化空白字符

    Args:
        text (str): 待清洗的文本。
    Returns:
        str: 清洗后的文本。
    """
    # 替换所有空白符（含小红书常见的全角空格\u3000、换行、制表符）
    text = re.sub(r'[\t\n\u3000]', ' ', text)
    # HTML标签移除
    text = re.sub(r'<[^>]+>', '', text)
    # URL移除, 适配中文域名
    text = remove_urls(text)
    # 全角转半角（保留全角￥）
    text = fullwidth_to_halfwidth(text)
    # 符号标准化
    text = normalize_punctuation(text)
    # 标准化合并连续空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
# apply the basic_clean function to the 'text' column
cont['text'] = cont['text'].apply(basic_clean)

In [12]:
cont['text'][6]

'[达拉斯.吃]快乐小羊,回到儿时澳门豆捞坊 Happy Lamb Hot Pot 📍 240 Legacy Dr ste 116, Plano, TX 75023 感谢快乐小羊的邀请 虽然不知道小羊快不快乐[笑哭R]但我吃的很快乐 特别喜欢小羊家环境,舒适低调不喧哗,适合朋友聊天 所有菜品自取,更自由快捷,绝大多数都很新鲜(只有个别鹌鹑蛋黄不知为何有点咸鸭蛋味道,也许是我敏感[害羞R])我们选的金汤酸辣锅和特制香辣锅 金汤锅其实更像酸菜锅,酸菜味挺浓,还有很麻的口感,下了鱼片和牛肉,秒变酸菜鱼和酸菜牛 香辣锅是台湾健康感麻辣小火锅味儿,相对重庆火锅更为清淡,也更能体现食材本身的味道 强烈推荐他家臻品羊肉片,肥瘦合适还有奶香味 然后手打羊肉丸,和香菜混合制作,特别好吃 台湾酸甜口包心菜泡菜永远吃不腻 作为碳水大户,炒饭炒面炸馒头炸麻球完全满足了我的需求,炸鸡翅和橙子也很不错,小朋友饭应有尽有~ 重点来了!小羊给的两杯鸡尾酒把不太喝酒的我给惊艳了!图10中矮的那杯清冽的酒精带着丝丝甜味,醇香口感干脆利落,会一直想喝不停 有柠檬片的那杯是一种混合果汁配着清淡酸奶泡的无酒精鸡尾酒,打败我爱的所有果汁 没想到小羊要靠鸡尾酒出道了[笑哭R] 一顿饭下来,不仅吃的满足,整个氛围也让人想起童年回忆里的澳门豆捞坊,干净惬意有情调 我就喜欢这样安安静静享受火锅的快乐 #达拉斯火锅[话题]# #达拉斯美食[话题]# #达拉斯生活[话题]# #达拉斯[话题]# #达拉斯探店[话题]# #达拉斯周边[话题]# #达拉斯周末[话题]# #达拉斯吃喝玩乐[话题]# #快乐小羊[话题]# #快乐小羊火锅[话题]#'

社交媒体特征处理层

In [ ]:
def social_media_clean(text:str) -> str:
    """
    社交媒体文本清洗，主要针对小红书平台的特定格式和符号进行处理。
    1. 移除话题标签（#）但保留关键词
    2. 移除@提及
    3. 转换表情符号（可选：移除或转换为文本描述）
    4. 处理小红书特有格式（如[笑哭R]）
    5. 去除多余空格

    Args:
        text (str): 待清洗的文本。
    Returns:
        str: 清洗后的文本。
    """

    # 移除话题标签但保留关键词（如 #达拉斯美食 → 达拉斯美食）
    text = re.sub(r'#([^\s#]+)', r'\1', text)
    
    # 移除@提及(包含其变体, 如@小红书用户)
    text = re.sub(r'@[\w\u4e00-\u9fff-]+', '', text)  # 支持中英文用户名
    
    # 转换表情符号（可选策略）
    strategy = 'remove'  # 或 'demojize'
    if strategy == 'remove':
        text = emoji.replace_emoji(text, replace='')
    else:
        text = emoji.demojize(text, delimiters=(' [', '] '))
    
    # 处理小红书特有格式（如[笑哭R]）
    text = re.sub(r'\[.*?\]', '', text)
    
    return text

'有没有和我一样【吃茶三千】一喝一个不吱声的 连地址都不想写了 虽然环境还不错，颜值也不错 但我感觉自己完全拔草了 尝了mango茶sample：仿佛化掉的芒果冰棍 冻顶乌龙：茶味白开水 又见柠檬挞：矮子里的将军，青柠汁绿茶+cream，虽然算清新派里能喝的，但依旧偏寡淡 松针厚奶：茶香精味重+依旧白开水 \t 是今天员工问题？还是达拉斯这家店问题？还是吃茶三千就这个风格？为什么值得排这么久的队呢…想不通啊想不通～'

中文英文混合语言环境优化层

In [ ]:
from langid import classify
import jieba

def language_optimize(text):
    # 语言检测（防止意外混入其他语言）
    lang, confidence = classify(text)
    if lang != 'zh' and confidence > 0.8:
        return ''  # 或标记为需人工审核
        
    # 中英文混合处理（保留有效英文术语）
    protected_terms = {'VIP', 'CEO', 'AI', 'NFT'}  # 自定义保护词表
    words = jieba.lcut(text)
    cleaned = []
    for word in words:
        if word.isalpha() and word.upper() not in protected_terms:
            # 非保护英文词转小写（可选）
            word = word.lower()  
        cleaned.append(word)
    
    return ' '.join(cleaned)

高级语义清洗层

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def semantic_clean(texts, ngram_range=(1,2), max_df=0.95, min_df=3):
    """
    基于TF-IDF的语义级清洗（需全量数据）
    """
    tfidf = TfidfVectorizer(ngram_range=ngram_range, 
                          max_df=max_df, 
                          min_df=min_df)
    matrix = tfidf.fit_transform(texts)
    
    # 识别高频无意义词（需自定义阈值）
    df = pd.DataFrame(matrix.toarray(), columns=tfidf.get_feature_names_out())
    term_freq = df.sum(axis=0)
    noise_terms = term_freq[term_freq > 500].index.tolist()  # 示例阈值
    
    # 构建正则过滤模式
    pattern = r'\b(?:{})\b'.format('|'.join(noise_terms))
    return [re.sub(pattern, '', text) for text in texts]

全流程集成示例

In [ ]:
class TextSanitizer:
    def __init__(self):
        self.emoji_dict = emoji.EMOJI_DATA  # 预加载表情符号库
        self.protected_terms = self._load_protected_terms()
        
    def _load_protected_terms(self):
        # 从文件/数据库加载保护词表（品牌词、产品词等）
        return {'鲜芋仙', 'MeetFresh', 'ParkPavilion'}  
    
    def pipeline(self, text):
        text = basic_clean(text)
        text = social_media_clean(text)
        text = language_optimize(text)
        return text
    
    def batch_process(self, texts):
        # 并行加速（利用全部CPU核心）
        from concurrent.futures import ProcessPoolExecutor
        with ProcessPoolExecutor() as executor:
            return list(executor.map(self.pipeline, texts))

In [ ]:
性能优化策略. 正则表达式预编译

In [ ]:
# 在类初始化时预编译正则表达式
class TextSanitizer:
    def __init__(self):
        self.url_pattern = re.compile(r'(https?://\S+)')
        self.mention_pattern = re.compile(r'@\w+')
        # ...其他正则预编译

In [ ]:
性能优化策略 多语言混合处理增强

In [ ]:
# 添加语言自动转换模块（需安装googletrans==4.0.0-rc1）
from googletrans import Translator

def translate_mixed_text(text):
    translator = Translator()
    chunks = []
    current_lang = 'zh'
    
    for sentence in re.split(r'([.!?]+\s*)', text):
        if not sentence.strip():
            continue
        detected = translator.detect(sentence).lang
        if detected != current_lang:
            translated = translator.translate(sentence, src=detected, dest='zh').text
            chunks.append(translated)
        else:
            chunks.append(sentence)
    
    return ''.join(chunks)

In [ ]:
性能优化策略 领域自适应清洗

In [ ]:
# 加载领域关键词白名单（餐饮类）
DOMAIN_KEYWORDS = set()
with open('food_terms.txt', encoding='utf-8') as f:
    DOMAIN_KEYWORDS.update(line.strip() for line in f)

def domain_aware_clean(text):
    words = jieba.lcut(text)
    return ' '.join([w for w in words if w in DOMAIN_KEYWORDS])

In [ ]:
质量评估指标

In [ ]:
def evaluate_clean_quality(original, cleaned):
    # 信息保留率
    orig_terms = set(jieba.lcut(original))
    cleaned_terms = set(jieba.lcut(cleaned))
    retention_rate = len(cleaned_terms & orig_terms) / len(orig_terms)
    
    # 噪声去除率
    noise_removal = 1 - (len(cleaned)/len(original))
    
    # 可读性评分（基于平均句长）
    sentences = re.split(r'[。！？]', cleaned)
    avg_len = sum(len(s) for s in sentences)/len(sentences)
    
    return {
        'retention': round(retention_rate, 3),
        'noise_removal': round(noise_removal, 3),
        'readability': 'good' if 15 < avg_len < 50 else 'poor'
    }

In [ ]:
# enable jieba , and use the accurate mode 
jieba.enable_paddle()
jieba.set_dictionary('..\..\Data\processed\dict.txt.big')
jieba.initialize()